In [1]:
import sys
import random
import collections
import queue
import argparse

In [2]:
DEFAULT_NUMBER_OF_TAXIS = 3
DEFAULT_END_TIME = 80
SEARCH_DURATION = 4
TRIP_DURATION = 10
DEPARTURE_INTERVAL = 5

In [3]:
Event = collections.namedtuple('Event', 'time proc action')

In [4]:
def compute_delay(interval):
    """Compute action delay using exponential distribution"""
    return int(random.expovariate(1/interval)) + 1

In [5]:
# BEGIN TAXI_PROCESS
def taxi_process(ident, trips, start_time=0):  # <1>
    """Yield to simulator issuing event at each state change"""
    time = yield Event(start_time, ident, 'leave garage')  # <2>
    for i in range(trips):  # <3>
        prowling_ends = time + compute_delay(SEARCH_DURATION)  # <4>
        time = yield Event(prowling_ends, ident, 'pick up passenger')  # <5>

        trip_ends = time + compute_delay(TRIP_DURATION)  # <6>
        time = yield Event(trip_ends, ident, 'drop off passenger')  # <7>

    yield Event(time + 1, ident, 'going home')  # <8>
    # end of taxi process # <9>
# END TAXI_PROCESS

In [6]:
# BEGIN TAXI_SIMULATOR
class Simulator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)


    def run(self, end_time):  # <1>
        """Schedule and display events until time is up"""
        # schedule the first event for each cab
        for _, proc in sorted(self.procs.items()):  # <2>
            first_event = next(proc)  # <3>
            self.events.put(first_event)  # <4>

        # main loop of the simulation
        time = 0
        while time < end_time:  # <5>
            if self.events.empty():  # <6>
                print('*** end of events ***')
                break

            # get and display current event
            current_event = self.events.get()  # <7>
            print('taxi:', current_event.proc,  # <8>
                  current_event.proc * '   ', current_event)

            # schedule next action for current proc
            time = current_event.time  # <9>
            proc = self.procs[current_event.proc]  # <10>
            try:
                next_event = proc.send(time)  # <11>
            except StopIteration:
                del self.procs[current_event.proc]  # <12>
            else:
                self.events.put(next_event)  # <13>
        else:  # <14>
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))
# END TAXI_SIMULATOR

In [7]:
def main(end_time=DEFAULT_END_TIME, num_taxis=DEFAULT_NUMBER_OF_TAXIS,
         seed=None):
    """Initialize random generator, build procs and run simulation"""
    if seed is not None:
        random.seed(seed)  # get reproducible results

    taxis = {i: taxi_process(i, (i+1)*2, i*DEPARTURE_INTERVAL)
             for i in range(num_taxis)}
    sim = Simulator(taxis)
    sim.run(end_time)

In [8]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser(
                        description='Taxi fleet simulator.')
    parser.add_argument('-e', '--end-time', type=int,
                        default=DEFAULT_END_TIME,
                        help='simulation end time; default = %s'
                        % DEFAULT_END_TIME)
    parser.add_argument('-t', '--taxis', type=int,
                        default=DEFAULT_NUMBER_OF_TAXIS,
                        help='number of taxis running; default = %s'
                        % DEFAULT_NUMBER_OF_TAXIS)
    parser.add_argument('-s', '--seed', type=int, default=None,
                        help='random generator seed (for testing)')

    args = parser.parse_args()
    main(args.end_time, args.taxis, args.seed)

usage: ipykernel_launcher.py [-h] [-e END_TIME] [-t TAXIS] [-s SEED]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Ivo\AppData\Roaming\jupyter\runtime\kernel-20ff8ff1-fd7b-4f8a-970f-b6d238b24c24.json


SystemExit: 2

C:\Users\Ivo\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3259: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### Taxi simulator
#### Sample run with two cars, random seed 10. This is a valid doctest.

In [9]:
>>> main(num_taxis=2, seed=10)

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 0  Event(time=4, proc=0, action='pick up passenger')
taxi: 1     Event(time=5, proc=1, action='leave garage')
taxi: 1     Event(time=9, proc=1, action='pick up passenger')
taxi: 0  Event(time=10, proc=0, action='drop off passenger')
taxi: 1     Event(time=12, proc=1, action='drop off passenger')
taxi: 0  Event(time=17, proc=0, action='pick up passenger')
taxi: 1     Event(time=19, proc=1, action='pick up passenger')
taxi: 1     Event(time=21, proc=1, action='drop off passenger')
taxi: 1     Event(time=24, proc=1, action='pick up passenger')
taxi: 0  Event(time=28, proc=0, action='drop off passenger')
taxi: 1     Event(time=28, proc=1, action='drop off passenger')
taxi: 0  Event(time=29, proc=0, action='going home')
taxi: 1     Event(time=30, proc=1, action='pick up passenger')
taxi: 1     Event(time=61, proc=1, action='drop off passenger')
taxi: 1     Event(time=62, proc=1, action='going home')
*** end of events ***


In [10]:
end_time=DEFAULT_END_TIME
num_taxis=2
seed=10
if seed is not None:
    random.seed(seed)  # get reproducible results

taxis = {i: taxi_process(i, (i+1)*2, i*DEPARTURE_INTERVAL)
         for i in range(num_taxis)}
sim = Simulator(taxis)
sim.run(end_time)

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 0  Event(time=4, proc=0, action='pick up passenger')
taxi: 1     Event(time=5, proc=1, action='leave garage')
taxi: 1     Event(time=9, proc=1, action='pick up passenger')
taxi: 0  Event(time=10, proc=0, action='drop off passenger')
taxi: 1     Event(time=12, proc=1, action='drop off passenger')
taxi: 0  Event(time=17, proc=0, action='pick up passenger')
taxi: 1     Event(time=19, proc=1, action='pick up passenger')
taxi: 1     Event(time=21, proc=1, action='drop off passenger')
taxi: 1     Event(time=24, proc=1, action='pick up passenger')
taxi: 0  Event(time=28, proc=0, action='drop off passenger')
taxi: 1     Event(time=28, proc=1, action='drop off passenger')
taxi: 0  Event(time=29, proc=0, action='going home')
taxi: 1     Event(time=30, proc=1, action='pick up passenger')
taxi: 1     Event(time=61, proc=1, action='drop off passenger')
taxi: 1     Event(time=62, proc=1, action='going home')
*** end of events ***
